In [1]:
import os
from skimage.io import imread_collection
import pathlib
import random
import time
import PIL
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from tensorflow.keras.regularizers import l1,l2
from LWTA.base import *

In [2]:
learning_rate = 0.003
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)  

meta_step_size = 0.25
inner_batch_size = 25
eval_batch_size = 25

meta_iters = 200000
eval_iters = 5
inner_iters = 4

eval_interval = 50
report_frequency = 50
checkpoint_freq = 1000
train_shots = 20
shots = 5 # the k for => k-shot N-way
classes = 5 # the N for => k-shot N-way
LWTA = True
BMA = False

In [3]:
# read images
train_images_names = []

for root, dirs, files in os.walk("mini-Imagenet/train"):
    for filename in files:
        train_images_names.append(filename)

test_images_names = []

for root, dirs, files in os.walk("mini-Imagenet/test"):
    for filename in files:
        test_images_names.append(filename)  

len(train_images_names), len(test_images_names)

(38400, 12000)

In [4]:
# create labels
train_labels = []

for image in train_images_names:
    train_labels.append(int(image[9:-4]))

test_labels = []

for image in test_images_names:
    test_labels.append(int(image[9:-4]))

train_labels[0:4], test_labels[0:4]

([615, 295, 153, 41], [938, 660, 1271, 1044])

In [5]:
# train_filenames = tf.constant(train_images_names)
# train_labels = tf.constant(train_labels)
# test_filenames = tf.constant(test_images_names)
# test_labels = tf.constant(test_labels)

# train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames, train_labels))
# test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames, test_labels))

# # def im_file_to_tensor(file, label):
# #     def _im_file_to_tensor(file, label):
# #         path = f"/mini-Imagenet/train/n01532829/{file.numpy().decode()}"
# #         im = tf.image.decode_jpeg(tf.io.read_file(path), channels=3)
# #         im = tf.cast(image_decoded, tf.float32) / 255.0
# #         return im, label
# #     return tf.py_function(_im_file_to_tensor, 
# #                           inp=(file, label), 
# #                           Tout=(tf.float32, tf.uint8))

# # train_dataset = train_dataset.map(im_file_to_tensor)

In [6]:
# train_ds_paths = tf.data.Dataset.list_files(str('mini-Imagenet/train/'+'*/*'))
# test_ds_paths = tf.data.Dataset.list_files(str('mini-Imagenet/test/'+'*/*'))

In [7]:
# def process_path(file_path):
#     label = tf.strings.split(file_path, os.sep)[-2]
#     return tf.io.read_file(file_path), label

# train_ds = train_ds_paths.map(process_path)
# train_ds

In [8]:
# train_data = []

# for x in train_ds_paths:
#     an_image = PIL.Image.open(x.numpy().decode("utf-8") )

#     image_sequence = an_image.getdata()
#     image_array = np.array(image_sequence)
#     dim_1 = dim_2 = int(np.sqrt(image_array.shape[0]))
#     image_array = image_array.reshape((dim_1, dim_2, channels))
#     train_data.append(image_array)

In [9]:
# test_data = []

# for x in test_ds_paths:
#     an_image = PIL.Image.open(x.numpy().decode("utf-8") )

#     image_sequence = an_image.getdata()
#     image_array = np.array(image_sequence)
#     dim_1 = dim_2 = int(np.sqrt(image_array.shape[0]))
#     image_array = image_array.reshape((dim_1, dim_2, channels))
#     test_data.append(image_array)

In [5]:
# with open('train_data.npy', 'wb') as f:
#     np.save(f, train_data)
    
# with open('test_data.npy', 'wb') as f:
#     np.save(f, test_data)

with open('train_data.npy', 'rb') as f:
    train_data = np.load(f)
    
with open('test_data.npy', 'rb') as f:
    test_data = np.load(f)

In [6]:
train_data = train_data.astype(np.uint8)
test_data = test_data.astype(np.uint8)

train_dataset_demo = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
test_dataset_demo = tf.data.Dataset.from_tensor_slices((test_data, test_labels))

In [7]:
class Dataset:
    # This class will facilitate the creation of a few-shot dataset
    # from the Mini-Imagenet dataset that can be sampled from quickly while also
    # allowing to create new labels at the same time.
    def __init__(self, training):
        # Download the tfrecord files containing the omniglot data and convert to a
        # dataset.
        if training:
            ds = train_dataset_demo
        else:
            ds = test_dataset_demo
        
        # Iterate over the dataset to get each individual image and its class,
        # and put that data into a dictionary.
        self.data = {}
        
        def extraction(image, label):           
            # This function will shrink the Omniglot images to the desired size,
            # scale pixel values and convert the RGB image to grayscale
            image = tf.image.convert_image_dtype(image, tf.float32)
            image = tf.image.resize(image, [84, 84])
            return image, label

        for image, label in ds.map(extraction):
            image = image.numpy()
            label = str(label.numpy())
            if label not in self.data:
                self.data[label] = []
            self.data[label].append(image)
            self.labels = list(self.data.keys())
       
    def get_mini_dataset(
        self, batch_size, repetitions, shots, num_classes, split=False
    ):
        temp_labels = np.zeros(shape=(num_classes * shots))
        temp_images = np.zeros(shape=(num_classes * shots, 84, 84, 3))
        if split:
            test_labels = np.zeros(shape=(num_classes))
            test_images = np.zeros(shape=(num_classes, 84, 84, 3))

        # Get a random subset of labels from the entire label set.
        label_subset = random.choices(self.labels, k=num_classes)
        for class_idx, class_obj in enumerate(label_subset):
            # Use enumerated index value as a temporary label for mini-batch in
            # few shot learning.
            temp_labels[class_idx * shots : (class_idx + 1) * shots] = class_idx
            # If creating a split dataset for testing, select an extra sample from each
            # label to create the test dataset.
            if split:
                test_labels[class_idx] = class_idx
                images_to_split = random.choices(
                    self.data[label_subset[class_idx]], k=shots + 1
                )
                test_images[class_idx] = images_to_split[-1]
                temp_images[
                    class_idx * shots : (class_idx + 1) * shots
                ] = images_to_split[:-1]
            else:
                # For each index in the randomly selected label_subset, sample the
                # necessary number of images.
                temp_images[
                    class_idx * shots : (class_idx + 1) * shots
                ] = random.choices(self.data[label_subset[class_idx]], k=shots)

        dataset = tf.data.Dataset.from_tensor_slices(
            (temp_images.astype(np.float32), temp_labels.astype(np.int32))
        )
        dataset = dataset.shuffle(100).batch(batch_size).repeat(repetitions)
        if split:
            return dataset, test_images, test_labels
        return dataset

import urllib3

urllib3.disable_warnings()  # Disable SSL warnings that may happen during download.
train_dataset = Dataset(training=True)
test_dataset = Dataset(training=False)

## Train the model (with dense_LWTA)

In [15]:
sb_class = LwtaClassifier(original_dim = [classes,1], tau = 5e-2, bma=BMA,dataset="Imagenet",
                          deterministic=False) 

def train_func(x, train=True, activation="lwta"):
    return sb_class(x, train=train, activation=activation)

train = tf.function(train_func)

In [19]:
# Create checkpoint for resuming training 
ckpt = tf.train.Checkpoint(step=tf.Variable(0), optimizer=optimizer, net=sb_class)
ckpt_path = f"./checkpoints_miniImagenet_lwta_{shots}_shot_{classes}_way"
manager = tf.train.CheckpointManager(ckpt, ckpt_path, max_to_keep=3)

def checkpoint_manager(meta_iter):  
    if meta_iter == 0:
        if os.path.isdir(ckpt_path):
            ckpt.restore(manager.latest_checkpoint)
            print("\nRestored from {}\n".format(manager.latest_checkpoint))
        else:
            print("\nNone checkpoints found => Initializing from scratch.\n")
    elif meta_iter % checkpoint_freq == 0:
        save_path = manager.save()
        print("\nSaved checkpoint for step {}: {}\n".format(meta_iter, save_path))
        
    ckpt.step.assign_add(1)
    return ckpt

In [20]:
!rm -r checkpoints_miniImagenet_1_shot_5_way
!rm -r checkpoints_miniImagenet_5_shot_5_way

rm: cannot remove 'checkpoints_miniImagenet_1_shot_5_way': No such file or directory
rm: cannot remove 'checkpoints_miniImagenet_5_shot_5_way': No such file or directory


In [22]:
training = []
testing = []

start = time.time()

for meta_iter in range(int(ckpt.step), meta_iters):

    mini_dataset = train_dataset.get_mini_dataset(
        inner_batch_size, inner_iters, train_shots, classes
    )
   
    ckpt = checkpoint_manager(meta_iter)

    frac_done = meta_iter / meta_iters
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    
    # Get a sample from the full dataset.
    if meta_iter > 0:
        old_vars = sb_class.get_weights()

    j = 0    
    for images, labels in mini_dataset:

        with tf.GradientTape() as tape:

            x1, x2, x3 = images.shape[0], images.shape[1], images.shape[2]
            images = tf.reshape(images, [x1, x2*x3*3])    
#             preds, _, _ = sb_class(images, train=True, activation="lwta")
            try:
                preds, _, _ = train(images, train=True, activation="lwta")
            except (UnboundLocalError, ValueError):
                train = tf.function(train_func)
                preds, _, _ = train(images, train=True, activation="lwta")
                
            if (j == 0) and (meta_iter == 0):
                old_vars = sb_class.get_weights()
            # we optimize the variational lower bound scaled by the number of data
            # points (so we can keep our intuitions about hyper-params such as the learning rate)
            #kl_loss = sum(sb_class.losses) / (x1 * x2)
            ce = keras.losses.sparse_categorical_crossentropy(labels, preds)
            #loss = ce + kl_loss
            loss = ce
           
        grads = tape.gradient(loss, sb_class.trainable_weights)
        optimizer.apply_gradients(zip(grads, sb_class.trainable_weights))
       
        j += 1

    new_vars = sb_class.get_weights()

    # Perform SGD for the meta step.
    for var in range(len(new_vars)):
        new_vars[var] = old_vars[var] + (
            (new_vars[var] - old_vars[var]) * cur_meta_step_size
        )
    # After the meta-learning step, reload the newly-trained weights into the model.
    sb_class.set_weights(new_vars)

    if meta_iter == 50:
        print("\n#### The first 50 iterations took {:.2f} secs ####".format(time.time()-start))
    
    # Evaluation loop
    if meta_iter % eval_interval == 0:
        accuracies = []
        for dataset in (train_dataset, test_dataset):
            # Sample a mini dataset from the full dataset.
            train_set, test_images, test_labels = dataset.get_mini_dataset(
                eval_batch_size, eval_iters, shots, classes, split=True
            )
            old_vars = sb_class.get_weights()
            # Train on the samples and get the resulting accuracies.
            for images, labels in train_set:
              
                with tf.GradientTape() as tape:
                    x1, x2, x3 = images.shape[0], images.shape[1], images.shape[2]
                    images = tf.reshape(images, [x1, x2*x3*3])
                    try:
                        preds, _, _ = train(images, train=True, activation="lwta")
                    except (UnboundLocalError, ValueError):
                        train = tf.function(train_func)
                        preds, _, _ = train(images, train=True, activation="lwta")
                    #kl_loss = sum(sb_class.losses) / (x1 * x2)
                    ce = keras.losses.sparse_categorical_crossentropy(labels, preds)
                    #loss = ce + kl_loss
                    loss = ce

                grads = tape.gradient(loss, sb_class.trainable_weights)
                optimizer.apply_gradients(zip(grads, sb_class.trainable_weights))
                
            x1, x2, x3 = test_images.shape[0], test_images.shape[1], test_images.shape[2]

            test_images = tf.reshape(test_images, [x1, x2*x3*3]) 
#             test_preds, _, _ = sb_class(test_images, train=False, activation="lwta") for bma=True
            try:
                test_preds, _, _ = train(test_images, train=False, activation="lwta")
            except (UnboundLocalError, ValueError):
                train = tf.function(train_func)
                test_preds, _, _ = train(test_images, train=False, activation="lwta")
                
            test_preds = tf.argmax(test_preds).numpy()
            num_correct = (test_preds == test_labels).sum()
            
            # Reset the weights after getting the evaluation accuracies.
            sb_class.set_weights(old_vars)
            accuracies.append(num_correct / classes)
      
        training.append(accuracies[0])
        testing.append(accuracies[1])
        
        if meta_iter % report_frequency == 0:
            print("Iter = %d => train_acc = %.2f%% / test_acc = %.2f%%" % (meta_iter,
                  100*np.mean(training),100*np.mean(testing)))

end = time.time()
print("The training took {:.2f} secs".format(end-start))

tf.Tensor(382261, shape=(), dtype=int32)


AssertionError: 

In [13]:
print("Final training accuracy = {:.1f} %%".format(100*np.mean(training)))
print("Final testing accuracy = {:.1f} %%".format(100*np.mean(testing)))

Final training accuracy = 31.9 %%
Final testing accuracy = 53.6 %%


## Train the model (without LWTA)

In [10]:
def conv_bn(x):
    x = layers.Conv2D(filters=64, kernel_size=3,  padding='same',
                      kernel_regularizer=l2(0.01),activity_regularizer=l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x)    
    return x

inputs = layers.Input(shape=(84, 84, 3))

x = conv_bn(inputs)
x = conv_bn(x)
x = conv_bn(x)
x = conv_bn(x)
x = layers.Flatten()(x)
outputs = layers.Dense(classes, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile()

In [9]:
def model_func(x):
    return model(x)

train = tf.function(model_func)

In [10]:
# Create checkpoint for resuming training 
ckpt = tf.train.Checkpoint(step=tf.Variable(0), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt, './checkpoints_miniImagenet', max_to_keep=3)

def checkpoint_manager(meta_iter):    
    if meta_iter == 0:
        if os.path.isdir("checkpoints_miniImagenet"):
            
            ckpt.restore(manager.latest_checkpoint)
            print("\nRestored from {}\n".format(manager.latest_checkpoint))
        else:
            print("\nNone checkpoints found => Initializing from scratch.\n")
    elif meta_iter % checkpoint_freq == 0:
        save_path = manager.save()
        print("\nSaved checkpoint for step {}: {}\n".format(meta_iter, save_path))
    
    ckpt.step.assign_add(1)

In [11]:
!rm -r checkpoints_miniImagenet_1_shot_5_way
!rm -r checkpoints_miniImagenet_5_shot_5_way

rm: cannot remove 'checkpoints_miniImagenet_1_shot_5_way': No such file or directory
rm: cannot remove 'checkpoints_miniImagenet_5_shot_5_way': No such file or directory


In [12]:
training = []
testing = []

start = time.time()

for meta_iter in range(int(ckpt.step), meta_iters):   
    mini_dataset = train_dataset.get_mini_dataset(
        inner_batch_size, inner_iters, train_shots, classes
    )
    
    checkpoint_manager(meta_iter)
        
    frac_done = meta_iter / meta_iters
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    # Temporarily save the weights from the model.
    old_vars = model.get_weights()
    # Get a sample from the full dataset.
    
    j = 0
    for images, labels in mini_dataset:
        with tf.GradientTape() as tape:
#             preds = model(images)
            try:
                preds = train(images)
            except (UnboundLocalError, ValueError):
                train = tf.function(model_func)
                preds = train(images)
            loss = keras.losses.sparse_categorical_crossentropy(labels, preds)
        grads = tape.gradient(loss, model.trainable_weights)

        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        j += 1
    new_vars = model.get_weights()
    # Perform SGD for the meta step.
    for var in range(len(new_vars)):
        new_vars[var] = old_vars[var] + (
            (new_vars[var] - old_vars[var]) * cur_meta_step_size
        )
    # After the meta-learning step, reload the newly-trained weights into the model.
    model.set_weights(new_vars)
    if meta_iter == 50:
        print("\n#### The first 50 iterations took {:.2f} secs ####".format(time.time()-start))
        
    # Evaluation loop
    if meta_iter % eval_interval == 0:
        accuracies = []
        for dataset in (train_dataset, test_dataset):
            # Sample a mini dataset from the full dataset.
            train_set, test_images, test_labels = dataset.get_mini_dataset(
                eval_batch_size, eval_iters, shots, classes, split=True
            )
            old_vars = model.get_weights()
            # Train on the samples and get the resulting accuracies.
            for images, labels in train_set:
                with tf.GradientTape() as tape:
#                     preds = model(images)
                    try:
                        preds = train(images)
                    except (UnboundLocalError, ValueError):
                        train = tf.function(model_func)
                        preds = train(images)
                    loss = keras.losses.sparse_categorical_crossentropy(labels, preds)                          
                grads = tape.gradient(loss, model.trainable_weights)
                optimizer.apply_gradients(zip(grads, model.trainable_weights))
            
            try:
                test_preds = train(test_images)
            except (UnboundLocalError, ValueError):
                train = tf.function(model_func)
                test_preds = train(test_images)
#             test_preds = model.predict(test_images)
            test_preds = tf.argmax(test_preds).numpy()
                        
            num_correct = (test_preds == test_labels).sum()
           
            # Reset the weights after getting the evaluation accuracies.
            model.set_weights(old_vars)
            accuracies.append(num_correct / classes)
            
        training.append(accuracies[0])
        testing.append(accuracies[1])
        
        if meta_iter % report_frequency == 0:
            print("Iter = %d => train_acc = %.2f%% / test_acc = %.2f%%" % (meta_iter,
                  100*np.mean(training),100*np.mean(testing)))

end = time.time()
print("The training took {:.2f} secs".format(end-start))


None checkpoints found => Initializing from scratch.

Iter = 0 => train_acc = 0.00% / test_acc = 80.00%

#### The first 50 iterations took 52.51 secs ####
Iter = 50 => train_acc = 27.06% / test_acc = 43.14%
Iter = 100 => train_acc = 28.12% / test_acc = 48.51%
Iter = 150 => train_acc = 28.87% / test_acc = 52.32%


KeyboardInterrupt: 

In [16]:
print("Final training accuracy = {:.1f} %%".format(100*np.mean(training)))
print("Final testing accuracy = {:.1f} %%".format(100*np.mean(testing)))

Final training accuracy = 23.4 %%
Final testing accuracy = 31.4 %%
